### Installing Required Libraries

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-2.11                  |         openblas          10 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-for

In [3]:
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


### Importing Required Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# to access wikipedia pages and perform required clenaup
from bs4 import BeautifulSoup
import lxml

import requests # library to handle requests

#import json # library to handle JSON files
##from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
###import matplotlib.cm as cm
###import matplotlib.colors as colors

# import k-means from clustering stage
###from sklearn.cluster import KMeans

###!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
###!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Accessing Wikipedia page to download Canada zip code data, perform required cleanup, transformation, and load data into pandas dataframe

In [8]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

### locate zipcode table from wikipedia page, review HTML content, and parse data to capture content and load into pandas dataframe

In [14]:
#<table class="wikitable sortable">
cnd_zip_table = soup.find('table',{'class':'wikitable sortable'})
cnd_zip_table

In [32]:
# identifying number of lines
row_marker = 0
for row in cnd_zip_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    if len(columns) > 0:
        row_marker += 1    

# creating dataframe to capture table details from web url
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

table_df = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'], index=range(0,row_marker))
#table_df.shape

# parsing data and loading into dataframe
row_marker = 0
for row in cnd_zip_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        myText=column.get_text()
        table_df.iat[row_marker,column_marker] = myText.strip('\t\r\n') # removing new line character
        column_marker = column_marker+ 1
    if len(columns) > 0:
        row_marker += 1

print('table_df shape :', table_df.shape)
table_df.head()

table_df shape : (287, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Perform required cleanup to obtain desired output
#### 1) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#### 2) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
#### 3) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [41]:
# step 1
temp_df = table_df[table_df['Borough']!='Not assigned']
print('temp_df shape :', temp_df.shape)

# step 2
temp_grouped_df = temp_df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
print('temp_grouped_df shape : ', temp_grouped_df.shape)
temp_grouped_df.head()

temp_df shape : (210, 3)
temp_grouped_df shape :  (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
# identify which postal code neighbourhood is having Not assigned value
temp_grouped_df[temp_grouped_df['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [44]:
# step 3
try:
    temp_grouped_df.at[temp_grouped_df[temp_grouped_df['Neighbourhood']=='Not assigned'].index[0],'Neighbourhood']=temp_grouped_df.at[temp_grouped_df[temp_grouped_df['Neighbourhood']=='Not assigned'].index[0],'Borough']
except:
    pass  # no change to data
#temp_grouped_df.ix[85]
'''
PostalCode                M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object
'''
temp_grouped_df.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [45]:
print('temp_grouped_df shape : ', temp_grouped_df.shape)

temp_grouped_df shape :  (103, 3)
